In [ ]:
pip install torch transformers datasets pandas tqdm


In [ ]:
import pandas as pd
from datasets import Dataset

# Load the dataset
dataset_path = "/usr/daily_mail_summarisation.csv"
df = pd.read_csv(dataset_path)

# Drop the 'url' column
df = df.drop(columns=["url"], errors="ignore")

# Rename columns
df = df.rename(columns={"article": "text", "highlights": "summary"})

# Convert to Hugging Face Dataset
hf_dataset = Dataset.from_pandas(df)

# Display a sample
print(hf_dataset)


Dataset({
    features: ['text', 'summary'],
    num_rows: 8176
})


In [ ]:
!pip install transformers datasets torch


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments
from datasets import load_dataset, Dataset


In [ ]:
import pandas as pd

# Load the CSV
df = pd.read_csv("/usr/daily_mail_summarisation.csv")

# Keep only relevant columns
df = df[['article', 'highlights']]  # Ensure correct column names

# Rename columns to 'text' and 'summary' for consistency
df.rename(columns={"article": "text", "highlights": "summary"}, inplace=True)

# Convert Pandas DataFrame to Hugging Face Dataset
hf_dataset = Dataset.from_pandas(df)


<ipython-input-10-01309038dabe>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={"article": "text", "highlights": "summary"}, inplace=True)


In [ ]:
from transformers import AutoTokenizer

model_name = "facebook/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(model_name)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
def preprocess_data(examples):
    # Tokenize the input text (article)
    model_inputs = tokenizer(
        examples["text"],  # The original article
        max_length=512,     # Limit the article length
        truncation=True,    # Cut off long articles
        padding="max_length" # Add padding to match model requirements
    )

    # Tokenize the summary (output labels)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["summary"],
            max_length=150,
            truncation=True,
            padding="max_length"
        )

    # Store the tokenized outputs as labels
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [ ]:
def preprocess_data(examples):
    print(examples)  # Check what data is being received
    return examples


In [ ]:
def preprocess_data(examples):
    inputs = tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)
    labels = tokenizer(examples["summary"], padding="max_length", truncation=True, max_length=128)

    inputs["labels"] = labels["input_ids"]
    return inputs


In [ ]:
def preprocess_data(examples):
    # Convert to strings and handle possible missing values
    inputs = [str(text) for text in examples["text"]]
    summaries = [str(summary) for summary in examples["summary"]]

    # Tokenize input text
    model_inputs = tokenizer(
        inputs,  # The article
        max_length=512,
        truncation=True,
        padding="max_length"
    )

    # Tokenize summary using text_target
    labels = tokenizer(
        text_target=summaries,
        max_length=150,
        truncation=True,
        padding="max_length"
    )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [ ]:
print(hf_dataset[386])


{'text': "Radio 1 listeners in shock as sex noises are played on Greg James' show: 'Were my ears playing tricks on me?'Dan Edgar and Ella Rae Wise put on a loved-up display during a sun-kissed trip to Bali ahead of the new series.", 'summary': 'Dan Edgar and Ella Rae Wise put on a loved-up display during a sun-kissed trip to Bali ahead of the new series.'}


In [ ]:
def preprocess_data(examples):
    # Convert input text and summary to lists of strings
    inputs = [str(text) for text in examples["text"]]
    summaries = [str(summary) for summary in examples["summary"]]

    # Tokenize input text
    model_inputs = tokenizer(
        inputs,
        max_length=512,
        truncation=True,
        padding="max_length"
    )

    # Tokenize summary (labels)
    labels = tokenizer(
        text_target=summaries,
        max_length=150,
        truncation=True,
        padding="max_length"
    )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [ ]:
def preprocess_data(examples):
    print(type(examples["text"]), type(examples["summary"]))  # Debugging step
    return tokenizer(examples["text"], truncation=True, padding="max_length")


In [ ]:
print(hf_dataset)


Dataset({
    features: ['text', 'summary'],
    num_rows: 8176
})


In [ ]:
print(type(tokenizer))


<class 'transformers.models.bart.tokenization_bart_fast.BartTokenizerFast'>


In [ ]:
import re

def clean_text(text):
    """Cleans text by removing extra spaces, handling special characters, and ensuring it's a string."""
    if isinstance(text, list):
        text = " ".join(text)  # Flatten nested lists into a single string

    if not isinstance(text, str):
        text = str(text)  # Convert any non-string types to string

    text = text.strip()  # Remove leading/trailing whitespace
    text = re.sub(r"\s+", " ", text)  # Replace multiple spaces/newlines with a single space
    return text

def preprocess_data(examples):
    # Debugging to check input format
    print("Example keys:", examples.keys())
    print("Example data:", examples["text"][:2])

    # Clean and flatten text inputs
    inputs = [clean_text(x) for x in examples["text"]]
    summaries = [clean_text(x) for x in examples["summary"]]

    # Tokenize input text
    model_inputs = tokenizer(
        inputs,
        truncation=True,
        padding="max_length",
        max_length=512
    )

    # Tokenize summary using text_target
    labels = tokenizer(
        text_target=summaries,
        truncation=True,
        padding="max_length",
        max_length=150
    )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [ ]:
tokenized_dataset = hf_dataset.map(preprocess_data, batched=True)


Map:   0%|          | 0/8176 [00:00<?, ? examples/s]

Example keys: KeysView({'text': ['Beyoncé showcases her incredible figure in plunging white dress with thigh-high leg slit as she promotes her new haircare brand CécredBeyoncé has shown off her flawless beauty in a new photo to promote her brand new haircare range.', "Radio 1 listeners in shock as sex noises are played on Greg James' show: 'Were my ears playing tricks on me?'BBC Radio 1 listeners were left choking on their cornflakes on Wednesday morning as sex noises were blared out on the airwaves. ", "TOWIE's Dan Edgar, 33, and Ella Rae Wise, 23, put on a loved-up display during trip as romance blossoms between the pair during trip to BaliDan Edgar and Ella Rae Wise put on a loved-up display during a sun-kissed trip to Bali ahead of the new series.", "Bradley Cooper recalls 'crazy' pitch meeting at Beyonce's home for a role in A Star Is Born and says her husband Jay-Z was 'watching Judge Judy'Bradley Cooper discussed the 'crazy' experience he had of meeting Beyonce and her husband J

In [ ]:
print(tokenized_dataset[0])  # Print the first example


{'text': 'Beyoncé showcases her incredible figure in plunging white dress with thigh-high leg slit as she promotes her new haircare brand CécredBeyoncé has shown off her flawless beauty in a new photo to promote her brand new haircare range.', 'summary': 'Beyoncé has shown off her flawless beauty in a new photo to promote her brand new haircare range.', 'input_ids': [0, 40401, 261, 12695, 22848, 69, 3997, 1955, 11, 18323, 154, 1104, 3588, 19, 18781, 12, 3530, 2985, 34915, 25, 79, 15885, 69, 92, 39799, 1322, 1518, 230, 1140, 438, 2050, 40401, 261, 12695, 34, 2343, 160, 69, 25257, 4002, 11, 10, 92, 1345, 7, 3720, 69, 1518, 92, 39799, 1322, 1186, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [ ]:
decoded_text = tokenizer.decode(tokenized_dataset[0]["input_ids"], skip_special_tokens=True)
print(decoded_text)


Beyoncé showcases her incredible figure in plunging white dress with thigh-high leg slit as she promotes her new haircare brand CécredBeyoncé has shown off her flawless beauty in a new photo to promote her brand new haircare range.


In [ ]:
!pip install evaluate


In [ ]:
from evaluate import load



In [ ]:
!pip install rouge_score


In [ ]:
from evaluate import load

# Load ROUGE metric
rouge = load("rouge")

# Example: Tokenized text and generated summary
reference_summary = ["This is the expected summary."]  # Replace with actual reference
generated_summary = ["This is the generated summary."]  # Replace with model output

# Compute ROUGE scores
scores = rouge.compute(predictions=generated_summary, references=reference_summary)

print("ROUGE Scores:", scores)


ROUGE Scores: {'rouge1': 0.8000000000000002, 'rouge2': 0.5, 'rougeL': 0.8000000000000002, 'rougeLsum': 0.8000000000000002}


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",            # Directory to save model checkpoints
    evaluation_strategy="epoch",       # Evaluate at the end of each epoch
    save_strategy="epoch",             # Save model at each epoch
    per_device_train_batch_size=8,     # Adjust based on GPU memory
    per_device_eval_batch_size=8,
    learning_rate=5e-5,
    weight_decay=0.01,                 # Helps prevent overfitting
    num_train_epochs=3,                # You can increase this if needed
    save_total_limit=2,                 # Only keep the last 2 checkpoints
    push_to_hub=False,
    logging_dir="./logs",               # Directory for logs
    logging_steps=500,
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
from transformers import AutoModelForSeq2SeqLM

model_name = "facebook/bart-large-cnn"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


In [ ]:
from datasets import DatasetDict

# 90% training, 10% validation
tokenized_dataset = tokenized_dataset.train_test_split(test_size=0.1)

# Now it has 'train' and 'validation' splits
print(tokenized_dataset)


DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 7358
    })
    test: Dataset({
        features: ['text', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 818
    })
})


In [ ]:
from transformers import DataCollatorForSeq2Seq

# Create a data collator for sequence-to-sequence tasks
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Update Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,
)


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [ ]:
trainer.train()

NameError: name 'trainer' is not defined